## Predicting 2023 MLB Season: 03 - Get + Merge Odds Data


This notebook merges odds data with our game data from the previous notebooks. I recieved the odds data from Scottfree Analytics. This data will be hidden from the repo to protect the rights of the paid-for data.

In [53]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',1000)

In [54]:
# Load in odds data
odds_df = pd.read_csv('./hidden_data/mlb_betting_odds.csv')
odds_df.head()

,season,date,away_team,away_score,away_point_spread,away_point_spread_line,away_money_line,home_team,home_score,home_point_spread,home_point_spread_line,home_money_line,over_under,over_line,under_line
0,2014,3/22/2014,los_angeles_dodgers,3,-1.5,-103,-170,arizona_diamondbacks,1,1.5,-115,150,6.5,-110,-110
1,2014,3/22/2014,los_angeles_dodgers,7,-1.5,132,-119,arizona_diamondbacks,5,1.5,-152,109,7.5,-110,-110
2,2014,3/30/2014,los_angeles_dodgers,1,-1.5,141,-115,san_diego_padres,3,1.5,-162,105,6.5,-110,-110
3,2014,3/31/2014,chicago_cubs,0,1.5,-128,170,pittsburgh_pirates,1,-1.5,100,-190,6.5,-110,-110
4,2014,3/31/2014,washington_nationals,9,-1.5,-118,-179,new_york_mets,7,1.5,-100,159,6.5,-110,-110


In [55]:
odds_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22920 entries, 0 to 22919
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   season                  22920 non-null  int64  
 1   date                    22920 non-null  object 
 2   away_team               22920 non-null  object 
 3   away_score              22920 non-null  int64  
 4   away_point_spread       22920 non-null  float64
 5   away_point_spread_line  22920 non-null  int64  
 6   away_money_line         22920 non-null  int64  
 7   home_team               22920 non-null  object 
 8   home_score              22920 non-null  int64  
 9   home_point_spread       22920 non-null  float64
 10  home_point_spread_line  22920 non-null  int64  
 11  home_money_line         22920 non-null  int64  
 12  over_under              22920 non-null  float64
 13  over_line               22920 non-null  int64  
 14  under_line              22920 non-null

In [56]:
# Change datetime format
odds_df['date_numeric'] = pd.to_datetime(odds_df.date).astype(str).str.replace('-','')
odds_df.head()

,season,date,away_team,away_score,away_point_spread,away_point_spread_line,away_money_line,home_team,home_score,home_point_spread,home_point_spread_line,home_money_line,over_under,over_line,under_line,date_numeric
0,2014,3/22/2014,los_angeles_dodgers,3,-1.5,-103,-170,arizona_diamondbacks,1,1.5,-115,150,6.5,-110,-110,20140322
1,2014,3/22/2014,los_angeles_dodgers,7,-1.5,132,-119,arizona_diamondbacks,5,1.5,-152,109,7.5,-110,-110,20140322
2,2014,3/30/2014,los_angeles_dodgers,1,-1.5,141,-115,san_diego_padres,3,1.5,-162,105,6.5,-110,-110,20140330
3,2014,3/31/2014,chicago_cubs,0,1.5,-128,170,pittsburgh_pirates,1,-1.5,100,-190,6.5,-110,-110,20140331
4,2014,3/31/2014,washington_nationals,9,-1.5,-118,-179,new_york_mets,7,1.5,-100,159,6.5,-110,-110,20140331


In [57]:
odds_df.away_team.unique()

array(['los_angeles_dodgers', 'chicago_cubs', 'washington_nationals',
       'atlanta_braves', 'st._louis_cardinals', 'colorado_rockies',
       'san_francisco_giants', 'kansas_city_royals', 'boston_red_sox',
       'toronto_blue_jays', 'minnesota_twins', 'cleveland_indians',
       'seattle_mariners', 'philadelphia_phillies', 'new_york_yankees',
       'arizona_diamondbacks', 'cincinnati_reds', 'san_diego_padres',
       'baltimore_orioles', 'chicago_white_sox', 'texas_rangers',
       'los_angeles_angels', 'milwaukee_brewers', 'oakland_athletics',
       'tampa_bay_rays', 'miami_marlins', 'new_york_mets',
       'pittsburgh_pirates', 'houston_astros', 'detroit_tigers',
       'cleveland_guardians', 'american_league', 'national_league'],
      dtype=object)

In [58]:
# manually remap team names with abbreviations for future joining
team_name_dict = {}
team_name_dict["philadelphia_phillies"]='PHI'
team_name_dict["san_diego_padres"]='SDN'
team_name_dict["san_francisco_giants"]='SFN'
team_name_dict["los_angeles_angels"]='ANA'
team_name_dict["detroit_tigers"]='DET'
team_name_dict["cincinnati_reds"]='CIN'
team_name_dict["new_york_yankees"]='NYA'
team_name_dict["texas_rangers"]='TEX'
team_name_dict["tampa_bay_rays"]='TBA'
team_name_dict["colorado_rockies"]='COL'
team_name_dict["minnesota_twins"]='MIN'
team_name_dict["kansas_city_royals"]='KCA'
team_name_dict["arizona_diamondbacks"]='ARI'
team_name_dict["baltimore_orioles"]='BAL'
team_name_dict["atlanta_braves"]='ATL'
team_name_dict["toronto_blue_jays"]='TOR'
team_name_dict["seattle_mariners"]='SEA'
team_name_dict["milwaukee_brewers"]='MIL'
team_name_dict["pittsburgh_pirates"]='PIT'
team_name_dict["new_york_mets"]='NYN'
team_name_dict["los_angeles_dodgers"]='LAN'
team_name_dict["oakland_athletics"]='OAK'
team_name_dict["washington_nationals"]='WAS'
team_name_dict["chicago_white_sox"]='CHA'
team_name_dict["st._louis_cardinals"]='SLN' 
team_name_dict["chicago_cubs"]='CHN'
team_name_dict["boston_red_sox"]='BOS'
team_name_dict["miami_marlins"]='MIA'
team_name_dict["houston_astros"]='HOU'
team_name_dict["cleveland_indians"]='CLE'
team_name_dict["cleveland_guardians"]='CLE'

In [59]:
# Create new columns mapping names with shorthand name
away_team_short = []
home_team_short = []

i = 0
for home_long, away_long in zip(odds_df['home_team'], odds_df['away_team']):
    if i%1000==0:
        print(i)
    try:
        home_short = team_name_dict[home_long]
        home_team_short.append(home_short)
    except KeyError:
        home_team_short.append(None)
    try:
        away_short = team_name_dict[away_long]
        away_team_short.append(away_short)
    except KeyError:
        away_team_short.append(None)
    i+=1 
    # break

odds_df['home_team_short'] = home_team_short
odds_df['away_team_short'] = away_team_short
odds_df.head()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000


,season,date,away_team,away_score,away_point_spread,away_point_spread_line,away_money_line,home_team,home_score,home_point_spread,home_point_spread_line,home_money_line,over_under,over_line,under_line,date_numeric,home_team_short,away_team_short
0,2014,3/22/2014,los_angeles_dodgers,3,-1.5,-103,-170,arizona_diamondbacks,1,1.5,-115,150,6.5,-110,-110,20140322,ARI,LAN
1,2014,3/22/2014,los_angeles_dodgers,7,-1.5,132,-119,arizona_diamondbacks,5,1.5,-152,109,7.5,-110,-110,20140322,ARI,LAN
2,2014,3/30/2014,los_angeles_dodgers,1,-1.5,141,-115,san_diego_padres,3,1.5,-162,105,6.5,-110,-110,20140330,SDN,LAN
3,2014,3/31/2014,chicago_cubs,0,1.5,-128,170,pittsburgh_pirates,1,-1.5,100,-190,6.5,-110,-110,20140331,PIT,CHN
4,2014,3/31/2014,washington_nationals,9,-1.5,-118,-179,new_york_mets,7,1.5,-100,159,6.5,-110,-110,20140331,NYN,WAS


In [60]:
# Load in game level data
game_df = pd.read_csv('output_data/input_model_data_01.csv')
game_df['date'] = game_df['date'].astype(str)
game_df.head(-5)

C:\Users\Dylan McGee\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\interactiveshell.py:3508: DtypeWarning: Columns (13,15,85,87) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead,BATAVG_162_h,BATAVG_162_v,OBP_162_h,OBP_162_v,SLG_162_h,SLG_162_v,OPS_162_h,OPS_162_v,SB_162_h,SB_162_v,CS_162_h,CS_162_v,ERR_162_h,ERR_162_v,SO_perc_162_h,SO_perc_162_v,BB_perc_162_h,BB_perc_162_v,BATAVG_30_h,BATAVG_30_v,OBP_30_h,OBP_30_v,SLG_30_h,SLG_30_v,OPS_30_h,OPS_30_v,SB_30_h,SB_30_v,CS_30_h,CS_30_v,ERR_30_h,ERR_30_v,SO_perc_30_h,SO_perc_30_v,BB_perc_30_h,BB_perc_30_v
0,20000329,0,Wed,CHN,NL,1,NYN,NL,1,5,3,54,N,NaN,NaN,NaN,TOK01,55000.0,206,100010210,001000020,33,12,1,0,2,5,1,0,1,10,0,5,1,0,4,0,13,3,3,3,0,0,27,10,2,0,1,0,33,7,1,0,1,3,1,1,0,3,0,4,0,0,1,0,8,5,5,5,1,0,27,12,0,0,4,0,marsr901,Randy Marsh,herna901,Angel Hernandez,fostm901,Marty Foster,kulpr901,Ron Kulpa,NaN,(none),NaN,(none),bayld001,Don Baylor,valeb102,Bobby Valentine,liebj001,Jon Lieber,hampm001,Mike Hampton,aguir001,Rick Aguilera,andrs001,Shane Andrews,liebj001,Jon Lieber,hampm001,Mike Hampton,youne001,Eric Young,4,bufod001,Damon Buford,8,gracm001,Mark Grace,3,sosas001,Sammy Sosa,9,rodrh001,Henry Rodriguez,7,andrs001,Shane Andrews,5,nievj002,Jose Nieves,6,giraj001,Joe Girardi,2,liebj001,Jon Lieber,1,hendr001,Rickey Henderson,7,hamid001,Darryl Hamilton,8,alfoe001,Edgardo Alfonzo,4,piazm001,Mike Piazza,2,ventr001,Robin Ventura,5,belld001,Derek Bell,9,zeilt001,Todd Zeile,3,ordor001,Rey Ordonez,6,hampm001,Mike Hampton,1,NaN,Y,2000,-2,0,8,200003290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20000330,0,Thu,NYN,NL,2,CHN,NL,2,5,1,66,N,NaN,NaN,NaN,TOK01,55000.0,235,00001000004,00001000000,37,6,2,0,1,5,1,1,1,8,0,5,1,0,0,0,10,5,0,0,0,0,33,14,2,0,2,0,36,5,0,0,0,0,2,0,0,6,1,9,0,0,2,0,10,7,5,5,0,0,33,14,0,0,0,0,herna901,Angel Hernandez,fostm901,Marty Foster,kulpr901,Ron Kulpa,marsr901,Randy Marsh,NaN,(none),NaN,(none),valeb102,Bobby Valentine,bayld001,Don Baylor,cookd001,Dennis Cook,yound002,Danny Young,NaN,(none),agbab001,Benny Agbayani,reedr002,Rick Reed,farnk001,Kyle Farnsworth,hendr001,Rickey Henderson,7,hamid001,Darryl Hamilton,8,alfoe001,Edgardo Alfonzo,4,piazm001,Mike Piazza,2,ventr001,Robin Ventura,5,belld001,Derek Bell,9,zeilt001,Todd Zeile,3,ordor001,Rey Ordonez,6,reedr002,Rick Reed,1,youne001,Eric Young,4,bufod001,Damon Buford,8,sosas001,Sammy Sosa,9,gracm001,Mark 

In [61]:
# Join odds and game data by date and teams 
merged_df = game_df.merge(odds_df, how = 'left', left_on = ['date', 'team_h', 'team_v'], right_on = ['date_numeric', 'home_team_short', 'away_team_short'])
merged_df.drop(['home_team_short', 'away_team_short', 'date_numeric', 'home_score', 'away_score', 'home_team', 'away_team', 'season_y', 'date_y'], axis = 1, inplace=True)
merged_df.head(-5)

,date_x,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season_x,run_diff,home_victory,run_total,date_dblhead,BATAVG_162_h,BATAVG_162_v,OBP_162_h,OBP_162_v,SLG_162_h,SLG_162_v,OPS_162_h,OPS_162_v,SB_162_h,SB_162_v,CS_162_h,CS_162_v,ERR_162_h,ERR_162_v,SO_perc_162_h,SO_perc_162_v,BB_perc_162_h,BB_perc_162_v,BATAVG_30_h,BATAVG_30_v,OBP_30_h,OBP_30_v,SLG_30_h,SLG_30_v,OPS_30_h,OPS_30_v,SB_30_h,SB_30_v,CS_30_h,CS_30_v,ERR_30_h,ERR_30_v,SO_perc_30_h,SO_perc_30_v,BB_perc_30_h,BB_perc_30_v,away_point_spread,away_point_spread_line,away_money_line,home_point_spread,home_point_spread_line,home_money_line,over_under,over_line,under_line
0,20000329,0,Wed,CHN,NL,1,NYN,NL,1,5,3,54,N,NaN,NaN,NaN,TOK01,55000.0,206,100010210,001000020,33,12,1,0,2,5,1,0,1,10,0,5,1,0,4,0,13,3,3,3,0,0,27,10,2,0,1,0,33,7,1,0,1,3,1,1,0,3,0,4,0,0,1,0,8,5,5,5,1,0,27,12,0,0,4,0,marsr901,Randy Marsh,herna901,Angel Hernandez,fostm901,Marty Foster,kulpr901,Ron Kulpa,NaN,(none),NaN,(none),bayld001,Don Baylor,valeb102,Bobby Valentine,liebj001,Jon Lieber,hampm001,Mike Hampton,aguir001,Rick Aguilera,andrs001,Shane Andrews,liebj001,Jon Lieber,hampm001,Mike Hampton,youne001,Eric Young,4,bufod001,Damon Buford,8,gracm001,Mark Grace,3,sosas001,Sammy Sosa,9,rodrh001,Henry Rodriguez,7,andrs001,Shane Andrews,5,nievj002,Jose Nieves,6,giraj001,Joe Girardi,2,liebj001,Jon Lieber,1,hendr001,Rickey Henderson,7,hamid001,Darryl Hamilton,8,alfoe001,Edgardo Alfonzo,4,piazm001,Mike Piazza,2,ventr001,Robin Ventura,5,belld001,Derek Bell,9,zeilt001,Todd Zeile,3,ordor001,Rey Ordonez,6,hampm001,Mike Hampton,1,NaN,Y,2000,-2,0,8,200003290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20000330,0,Thu,NYN,NL,2,CHN,NL,2,5,1,66,N,NaN,NaN,NaN,TOK01,55000.0,235,00001000004,00001000000,37,6,2,0,1,5,1,1,1,8,0,5,1,0,0,0,10,5,0,0,0,0,33,14,2,0,2,0,36,5,0,0,0,0,2,0,0,6,1,9,0,0,2,0,10,7,5,5,0,0,33,14,0,0,0,0,herna901,Angel Hernandez,fostm901,Marty Foster,kulpr901,Ron Kulpa,marsr901,Randy Marsh,NaN,(none),NaN,(none),valeb102,Bobby Valentine,bayld001,Don Baylor,cookd001,Dennis Cook,yound002,Danny Young,NaN,(none),agbab001,Benny Agbayani,reedr002,Rick Reed,farnk001,Kyle Farnsworth,hendr001,Rickey Henderson,7,hamid001,Darryl Hamilton,8,alfoe001,Edgardo Alfonzo,4,piazm001,Mike Piazza,2,ventr001,

In [62]:
# Remove games with no odds
no_nans = merged_df[~merged_df['home_point_spread'].isna()]
no_nans.rename(columns={'date_x':'date'}, inplace= True)

no_nans.head()

C:\Users\Dylan McGee\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\core\frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season_x,run_diff,home_victory,run_total,date_dblhead,BATAVG_162_h,BATAVG_162_v,OBP_162_h,OBP_162_v,SLG_162_h,SLG_162_v,OPS_162_h,OPS_162_v,SB_162_h,SB_162_v,CS_162_h,CS_162_v,ERR_162_h,ERR_162_v,SO_perc_162_h,SO_perc_162_v,BB_perc_162_h,BB_perc_162_v,BATAVG_30_h,BATAVG_30_v,OBP_30_h,OBP_30_v,SLG_30_h,SLG_30_v,OPS_30_h,OPS_30_v,SB_30_h,SB_30_v,CS_30_h,CS_30_v,ERR_30_h,ERR_30_v,SO_perc_30_h,SO_perc_30_v,BB_perc_30_h,BB_perc_30_v,away_point_spread,away_point_spread_line,away_money_line,home_point_spread,home_point_spread_line,home_money_line,over_under,over_line,under_line
34011,20140322,0,Sat,LAN,NL,1,ARI,NL,1,3,1,54,N,NaN,NaN,NaN,SYD01,38266.0,169,10200000,000001000,33,5,2,0,1,3,0,0,1,3,0,11,0,0,0,0,7,4,1,1,1,0,27,13,1,0,0,0,33,5,1,0,0,1,0,0,0,2,0,10,0,0,0,0,7,5,3,3,1,0,27,10,1,0,0,0,welkt901,Tim Welke,scotd901,Dale Scott,diazl901,Laz Diaz,carlm901,Mark Carlson,NaN,(none),NaN,(none),mattd001,Don Mattingly,gibsk001,Kirk Gibson,kersc001,Clayton Kershaw,milew001,Wade Miley,jansk001,Kenley Jansen,ethia001,Andre Ethier,kersc001,Clayton Kershaw,milew001,Wade Miley,puigy001,Yasiel Puig,9,turnj001,Justin Turner,4,ramih003,Hanley Ramirez,6,gonza003,Adrian Gonzalez,3,vanss001,Scott Van Slyke,7,uribj002,Juan Uribe,5,ethia001,Andre Ethier,8,ellia001,A.J. Ellis,2,kersc001,Clayton Kershaw,1,polla001,A.J. Pollock,8,hilla001,Aaron Hill,4,goldp001,Paul Goldschmidt,3,pradm001,Martin Prado,5,trumm001,Mark Trumbo,7,montm001,Miguel Montero,2,owinc001,Chris Owings,6,parrg001,Gerardo Parra,9,milew001,Wade Miley,1,NaN,Y,2014,-2,0,4,201403220,0.258633,0.263522,0.320743,0.322272,0.391473,0.396285,0.712215,0.718557,62.0,78.0,41.0,28.0,75.0,109.0,0.201198,0.208705,0.091438,0.086687,0.253644,0.256335,0.313673,0.311372,0.377065,0.414230,0.690738,0.725602,14.0,12.0,6.0,8.0,11.0,16.0,0.190476,0.222222,0.087464,0.079922,-1.5,-103.0,-170.0,1.5,-115.0,150.0,6.5,-110.0,-110.0
34012,20140322,0,Sat,LAN,NL,1,ARI,NL,1,3,1,54,N,NaN,NaN,NaN,SYD01,38266.0,169,10200000,000001000,33,5,2,0,1,3,0,0,1,3,0,11,0,0,0,0,7,4,1,1,1,0,27,13,1,0,0,0,33,5,1,0,0,1,0,0,0,2,0,10,0,0,0,0,7,5,3,3,1,0,27,10,1,0,0,0,welkt901,Tim Welke,scotd901,Dale Scott,diazl901,Laz Diaz,carlm901,Mark Carlson,NaN,(none),NaN,(none),mattd001,Don Mattingly,gibsk001,Kirk Gibson,kersc001,Clayton Kershaw,milew001,Wade Miley,jansk001,Kenley Jansen,ethia001,Andre Ethier,

In [68]:
# Add the implied win probability as columns
def line_to_prob(line):
    prob_underdog = 100/(np.abs(line)+100) # this is the probability for the 
    add_term = ((1-np.sign(line))/2) # 0 if negative, 1 if positive
    mult_factor = np.sign(line) # -1 if negative, 1 if positive
    # if line is positive, team is underdog, give 0 + 1*prob_underdog
    # if line is negative, team is favoritesm give 1 + (-1)*prob_underdog
    imp_prob = add_term + mult_factor * prob_underdog 
    return(imp_prob)

implied_prob_h_ls = []
implied_prob_v_ls = []

for home_ml, away_ml in zip(no_nans['home_money_line'], no_nans['away_money_line']):
    implied_prob_h_ls.append(line_to_prob(home_ml))
    implied_prob_v_ls.append(line_to_prob(away_ml))

no_nans['implied_prob_h'] = implied_prob_h_ls
no_nans['implied_prob_v'] = implied_prob_v_ls

no_nans.head()

C:\Users\Dylan McGee\AppData\Local\Temp\ipykernel_17808\283457598.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_nans['implied_prob_h'] = implied_prob_h_ls
C:\Users\Dylan McGee\AppData\Local\Temp\ipykernel_17808\283457598.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_nans['implied_prob_v'] = implied_prob_v_ls


,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season_x,run_diff,home_victory,run_total,date_dblhead,BATAVG_162_h,BATAVG_162_v,OBP_162_h,OBP_162_v,SLG_162_h,SLG_162_v,OPS_162_h,OPS_162_v,SB_162_h,SB_162_v,CS_162_h,CS_162_v,ERR_162_h,ERR_162_v,SO_perc_162_h,SO_perc_162_v,BB_perc_162_h,BB_perc_162_v,BATAVG_30_h,BATAVG_30_v,OBP_30_h,OBP_30_v,SLG_30_h,SLG_30_v,OPS_30_h,OPS_30_v,SB_30_h,SB_30_v,CS_30_h,CS_30_v,ERR_30_h,ERR_30_v,SO_perc_30_h,SO_perc_30_v,BB_perc_30_h,BB_perc_30_v,away_point_spread,away_point_spread_line,away_money_line,home_point_spread,home_point_spread_line,home_money_line,over_under,over_line,under_line,implied_prob_h,implied_prob_v
34011,20140322,0,Sat,LAN,NL,1,ARI,NL,1,3,1,54,N,NaN,NaN,NaN,SYD01,38266.0,169,10200000,000001000,33,5,2,0,1,3,0,0,1,3,0,11,0,0,0,0,7,4,1,1,1,0,27,13,1,0,0,0,33,5,1,0,0,1,0,0,0,2,0,10,0,0,0,0,7,5,3,3,1,0,27,10,1,0,0,0,welkt901,Tim Welke,scotd901,Dale Scott,diazl901,Laz Diaz,carlm901,Mark Carlson,NaN,(none),NaN,(none),mattd001,Don Mattingly,gibsk001,Kirk Gibson,kersc001,Clayton Kershaw,milew001,Wade Miley,jansk001,Kenley Jansen,ethia001,Andre Ethier,kersc001,Clayton Kershaw,milew001,Wade Miley,puigy001,Yasiel Puig,9,turnj001,Justin Turner,4,ramih003,Hanley Ramirez,6,gonza003,Adrian Gonzalez,3,vanss001,Scott Van Slyke,7,uribj002,Juan Uribe,5,ethia001,Andre Ethier,8,ellia001,A.J. Ellis,2,kersc001,Clayton Kershaw,1,polla001,A.J. Pollock,8,hilla001,Aaron Hill,4,goldp001,Paul Goldschmidt,3,pradm001,Martin Prado,5,trumm001,Mark Trumbo,7,montm001,Miguel Montero,2,owinc001,Chris Owings,6,parrg001,Gerardo Parra,9,milew001,Wade Miley,1,NaN,Y,2014,-2,0,4,201403220,0.258633,0.263522,0.320743,0.322272,0.391473,0.396285,0.712215,0.718557,62.0,78.0,41.0,28.0,75.0,109.0,0.201198,0.208705,0.091438,0.086687,0.253644,0.256335,0.313673,0.311372,0.377065,0.414230,0.690738,0.725602,14.0,12.0,6.0,8.0,11.0,16.0,0.190476,0.222222,0.087464,0.079922,-1.5,-103.0,-170.0,1.5,-115.0,150.0,6.5,-110.0,-110.0,0.400000,0.629630
34012,20140322,0,Sat,LAN,NL,1,ARI,NL,1,3,1,54,N,NaN,NaN,NaN,SYD01,38266.0,169,10200000,000001000,33,5,2,0,1,3,0,0,1,3,0,11,0,0,0,0,7,4,1,1,1,0,27,13,1,0,0,0,33,5,1,0,0,1,0,0,0,2,0,10,0,0,0,0,7,5,3,3,1,0,27,10,1,0,0,0,welkt901,Tim Welke,scotd901,Dale Scott,diazl901,Laz Diaz,carlm901,Mark Carlson,NaN,(none),NaN,(none),mattd001,Don Mattingly,gibsk001,Kirk Gibson,kersc001,Clayton Kershaw,milew001,Wade Mil

In [71]:
# Add implied probablity mid point as source of truth
imp_prob_mid = []
for h_prob, v_prob in zip(no_nans['implied_prob_h'], no_nans['implied_prob_v']):
    print(h_prob, v_prob)
    try:
        imp_prob_mid.append((float(h_prob) + (1-float(v_prob)))/2)
    except TypeError:
        imp_prob_mid.append(None)

no_nans['implied_prob_h_mid'] = imp_prob_mid
no_nans.head()

0.4 0.6296296296296297
0.4784688995215311 0.54337899543379
0.4878048780487805 0.5348837209302326
0.5327102803738317 0.49019607843137253
0.4878048780487805 0.5348837209302326
0.6363636363636364 0.39215686274509803
0.5967741935483871 0.4291845493562232
0.5918367346938775 0.43478260869565216
0.6212121212121212 0.40160642570281124
0.5121951219512195 0.5121951219512195
0.4807692307692308 0.5412844036697247
0.47619047619047616 0.5454545454545454
0.5918367346938775 0.43478260869565216
0.49261083743842365 0.5305164319248826
0.3861003861003861 0.6415770609318996
0.6551724137931034 0.37037037037037035
0.6296296296296297 0.4
0.425531914893617 0.6
0.6226415094339622 0.4
0.5934959349593496 0.4329004329004329
0.5391705069124424 0.4830917874396135
0.4716981132075472 0.5495495495495495
0.5327102803738317 0.49019607843137253
0.425531914893617 0.6
0.5867768595041323 0.44052863436123346
0.4830917874396135 0.5391705069124424
0.5833333333333333 0.4444444444444444
0.6666666666666667 0.37037037037037035
0.4 

C:\Users\Dylan McGee\AppData\Local\Temp\ipykernel_17808\3004923721.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_nans['implied_prob_h_mid'] = imp_prob_mid


,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season_x,run_diff,home_victory,run_total,date_dblhead,BATAVG_162_h,BATAVG_162_v,OBP_162_h,OBP_162_v,SLG_162_h,SLG_162_v,OPS_162_h,OPS_162_v,SB_162_h,SB_162_v,CS_162_h,CS_162_v,ERR_162_h,ERR_162_v,SO_perc_162_h,SO_perc_162_v,BB_perc_162_h,BB_perc_162_v,BATAVG_30_h,BATAVG_30_v,OBP_30_h,OBP_30_v,SLG_30_h,SLG_30_v,OPS_30_h,OPS_30_v,SB_30_h,SB_30_v,CS_30_h,CS_30_v,ERR_30_h,ERR_30_v,SO_perc_30_h,SO_perc_30_v,BB_perc_30_h,BB_perc_30_v,away_point_spread,away_point_spread_line,away_money_line,home_point_spread,home_point_spread_line,home_money_line,over_under,over_line,under_line,implied_prob_h,implied_prob_v,implied_prob_h_mid
34011,20140322,0,Sat,LAN,NL,1,ARI,NL,1,3,1,54,N,NaN,NaN,NaN,SYD01,38266.0,169,10200000,000001000,33,5,2,0,1,3,0,0,1,3,0,11,0,0,0,0,7,4,1,1,1,0,27,13,1,0,0,0,33,5,1,0,0,1,0,0,0,2,0,10,0,0,0,0,7,5,3,3,1,0,27,10,1,0,0,0,welkt901,Tim Welke,scotd901,Dale Scott,diazl901,Laz Diaz,carlm901,Mark Carlson,NaN,(none),NaN,(none),mattd001,Don Mattingly,gibsk001,Kirk Gibson,kersc001,Clayton Kershaw,milew001,Wade Miley,jansk001,Kenley Jansen,ethia001,Andre Ethier,kersc001,Clayton Kershaw,milew001,Wade Miley,puigy001,Yasiel Puig,9,turnj001,Justin Turner,4,ramih003,Hanley Ramirez,6,gonza003,Adrian Gonzalez,3,vanss001,Scott Van Slyke,7,uribj002,Juan Uribe,5,ethia001,Andre Ethier,8,ellia001,A.J. Ellis,2,kersc001,Clayton Kershaw,1,polla001,A.J. Pollock,8,hilla001,Aaron Hill,4,goldp001,Paul Goldschmidt,3,pradm001,Martin Prado,5,trumm001,Mark Trumbo,7,montm001,Miguel Montero,2,owinc001,Chris Owings,6,parrg001,Gerardo Parra,9,milew001,Wade Miley,1,NaN,Y,2014,-2,0,4,201403220,0.258633,0.263522,0.320743,0.322272,0.391473,0.396285,0.712215,0.718557,62.0,78.0,41.0,28.0,75.0,109.0,0.201198,0.208705,0.091438,0.086687,0.253644,0.256335,0.313673,0.311372,0.377065,0.414230,0.690738,0.725602,14.0,12.0,6.0,8.0,11.0,16.0,0.190476,0.222222,0.087464,0.079922,-1.5,-103.0,-170.0,1.5,-115.0,150.0,6.5,-110.0,-110.0,0.400000,0.629630,0.385185
34012,20140322,0,Sat,LAN,NL,1,ARI,NL,1,3,1,54,N,NaN,NaN,NaN,SYD01,38266.0,169,10200000,000001000,33,5,2,0,1,3,0,0,1,3,0,11,0,0,0,0,7,4,1,1,1,0,27,13,1,0,0,0,33,5,1,0,0,1,0,0,0,2,0,10,0,0,0,0,7,5,3,3,1,0,27,10,1,0,0,0,welkt901,Tim Welke,scotd901,Dale Scott,diazl901,Laz Diaz,carlm901,Mark Carlson,NaN,(none),NaN,(none),mattd001,Don Mattingly,gibsk001,Kirk Gibson,kersc001,Clayt

In [72]:
# Add O/U result - if run_total > over_under, O...
o_u_results = []
for total, pred in zip(no_nans['run_total'], no_nans['over_under']):
    if total > pred:
        o_u_results.append('O')
    elif total < pred:
        o_u_results.append('U')
    elif total == pred:
        o_u_results.append('N')

no_nans['over_under_result'] = o_u_results

no_nans.head(20)

4 6.5
4 7.5
4 6.5
13 7.0
3 9.0
8 7.0
7 7.0
2 7.0
11 7.0
24 7.5
17 8.0
1 7.0
11 6.5
2 8.0
16 6.5
1 6.5
11 7.5
8 8.5
6 7.5
5 8.5
9 8.5
7 7.0
7 8.5
5 6.5
10 7.5
8 8.5
13 8.0
3 7.5
4 8.0
7 7.5
7 8.0
10 7.5
10 8.0
3 8.0
7 9.5
2 8.5
1 7.0
11 7.5
1 9.0
6 6.5
7 7.0
6 6.5
7 9.0
19 7.5
6 8.5
5 8.0
9 8.0
13 8.5
13 7.5
13 7.5
10 7.0
5 7.0
8 8.5
9 7.5
14 7.5
12 9.5
12 8.5
9 8.5
10 8.5
9 11.0
14 10.0
12 7.5
10 7.5
7 7.0
14 7.0
3 7.5
13 8.5
10 8.5
13 8.5
6 9.0
7 8.0
4 7.0
9 7.5
4 9.0
2 6.5
13 9.5
9 7.5
5 6.5
9 6.5
7 7.0
8 6.5
4 9.0
17 7.5
4 7.5
11 8.0
6 7.0
9 7.5
3 7.0
10 8.5
11 7.5
8 9.0
8 6.5
6 7.5
3 7.0
3 7.0
3 7.0
6 9.0
10 8.5
6 7.5
11 8.0
6 8.0
9 9.5
8 6.5
17 9.5
14 7.5
1 7.5
19 9.0
8 7.5
7 9.5
4 7.0
13 7.0
18 9.5
5 7.0
14 9.0
10 7.5
12 7.0
5 7.0
6 8.5
2 8.0
2 8.5
3 8.0
3 8.5
10 9.0
11 8.5
9 8.0
2 8.5
10 9.0
7 7.0
12 9.0
14 10.0
13 7.0
13 7.5
10 7.5
4 7.5
17 7.0
10 8.0
7 8.5
5 8.5
10 9.5
10 7.5
9 7.5
8 7.0
11 8.0
8 7.0
9 8.0
2 8.5
15 7.5
11 8.0
6 8.0
10 6.5
1 8.0
6 8.5
13 7.5
3 7.0
6 8.5
9 7.0
6

C:\Users\Dylan McGee\AppData\Local\Temp\ipykernel_17808\2656832346.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_nans['over_under_result'] = o_u_results


,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season_x,run_diff,home_victory,run_total,date_dblhead,BATAVG_162_h,BATAVG_162_v,OBP_162_h,OBP_162_v,SLG_162_h,SLG_162_v,OPS_162_h,OPS_162_v,SB_162_h,SB_162_v,CS_162_h,CS_162_v,ERR_162_h,ERR_162_v,SO_perc_162_h,SO_perc_162_v,BB_perc_162_h,BB_perc_162_v,BATAVG_30_h,BATAVG_30_v,OBP_30_h,OBP_30_v,SLG_30_h,SLG_30_v,OPS_30_h,OPS_30_v,SB_30_h,SB_30_v,CS_30_h,CS_30_v,ERR_30_h,ERR_30_v,SO_perc_30_h,SO_perc_30_v,BB_perc_30_h,BB_perc_30_v,away_point_spread,away_point_spread_line,away_money_line,home_point_spread,home_point_spread_line,home_money_line,over_under,over_line,under_line,implied_prob_h,implied_prob_v,implied_prob_h_mid,over_under_result
34011,20140322,0,Sat,LAN,NL,1,ARI,NL,1,3,1,54,N,NaN,NaN,NaN,SYD01,38266.0,169,10200000,000001000,33,5,2,0,1,3,0,0,1,3,0,11,0,0,0,0,7,4,1,1,1,0,27,13,1,0,0,0,33,5,1,0,0,1,0,0,0,2,0,10,0,0,0,0,7,5,3,3,1,0,27,10,1,0,0,0,welkt901,Tim Welke,scotd901,Dale Scott,diazl901,Laz Diaz,carlm901,Mark Carlson,NaN,(none),NaN,(none),mattd001,Don Mattingly,gibsk001,Kirk Gibson,kersc001,Clayton Kershaw,milew001,Wade Miley,jansk001,Kenley Jansen,ethia001,Andre Ethier,kersc001,Clayton Kershaw,milew001,Wade Miley,puigy001,Yasiel Puig,9,turnj001,Justin Turner,4,ramih003,Hanley Ramirez,6,gonza003,Adrian Gonzalez,3,vanss001,Scott Van Slyke,7,uribj002,Juan Uribe,5,ethia001,Andre Ethier,8,ellia001,A.J. Ellis,2,kersc001,Clayton Kershaw,1,polla001,A.J. Pollock,8,hilla001,Aaron Hill,4,goldp001,Paul Goldschmidt,3,pradm001,Martin Prado,5,trumm001,Mark Trumbo,7,montm001,Miguel Montero,2,owinc001,Chris Owings,6,parrg001,Gerardo Parra,9,milew001,Wade Miley,1,NaN,Y,2014,-2,0,4,201403220,0.258633,0.263522,0.320743,0.322272,0.391473,0.396285,0.712215,0.718557,62.0,78.0,41.0,28.0,75.0,109.0,0.201198,0.208705,0.091438,0.086687,0.253644,0.256335,0.313673,0.311372,0.377065,0.414230,0.690738,0.725602,14.0,12.0,6.0,8.0,11.0,16.0,0.190476,0.222222,0.087464,0.079922,-1.5,-103.0,-170.0,1.5,-115.0,150.0,6.5,-110.0,-110.0,0.400000,0.629630,0.385185,U
34012,20140322,0,Sat,LAN,NL,1,ARI,NL,1,3,1,54,N,NaN,NaN,NaN,SYD01,38266.0,169,10200000,000001000,33,5,2,0,1,3,0,0,1,3,0,11,0,0,0,0,7,4,1,1,1,0,27,13,1,0,0,0,33,5,1,0,0,1,0,0,0,2,0,10,0,0,0,0,7,5,3,3,1,0,27,10,1,0,0,0,welkt901,Tim Welke,scotd901,Dale Scott,diazl901,Laz Diaz,carlm901,Mark Carlson,NaN,(none),NaN,(none),mattd001,Don Mattingly,gibsk001,Kirk G

In [73]:
no_nans.shape

(22801, 215)

In [74]:
no_nans.to_csv('./hidden_data/mlb_game_and_odds.csv', index = False)